# Analyse de l'Impact des Conditions Climatiques sur la Sinistralité Automobile

## Contexte du Projet

Ce projet vise à analyser l'**impact des conditions climatiques sur la sinistralité automobile** en utilisant des méthodes de réduction de dimension et de machine learning.

### Objectifs

1. Construction d'une base jointe (assurance × climat)
2. Définition des variables cibles (fréquence et gravité)
3. Analyse descriptive complète
4. Réduction de dimension (ACP/PLS)
5. Modélisation prédictive
6. Interprétation et conclusions

In [ ]:
# Import des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Import des modules personnalisés
import sys
sys.path.append('..')
from src import data_preprocessing as dp
from src import feature_engineering as fe
from src import dimension_reduction as dr
from src import models as md
from src import evaluation as ev

print("Bibliothèques chargées avec succès!")

## 1. Chargement et Préparation des Données

### 1.1 Chargement des bases de données

In [ ]:
# Chargement de toutes les données
donnees = dp.charger_toutes_donnees('..')

df_polices = donnees['polices']
df_sinistres = donnees['sinistres']
df_climat = donnees['climat']
df_communes = donnees['communes']

print("\nAperçu des polices:")
print(df_polices.head())

### 1.2 Nettoyage et préparation

In [ ]:
# Nettoyer les polices
df_polices_clean = dp.nettoyer_donnees_polices(df_polices)

# Nettoyer les sinistres
df_sinistres_clean = dp.nettoyer_donnees_sinistres(df_sinistres)

# Agréger les sinistres par police
df_sinistres_agg = dp.agregation_sinistres_par_police(df_sinistres_clean)

print("\nStatistiques des sinistres agrégés:")
print(df_sinistres_agg.describe())

### 1.3 Construction de la base finale

In [ ]:
# Joindre polices et sinistres
df_base = fe.joindre_polices_sinistres(df_polices_clean, df_sinistres_agg)

# Préparer les données climatiques
df_climat_prep, variables_climat = dp.preparer_variables_climatiques(df_climat)
df_climat_agg = fe.agregation_climat_par_dept_annee(df_climat_prep, variables_climat)

# Joindre avec les données climatiques
df_finale = fe.joindre_avec_climat(df_base, df_climat_agg)

# Créer des variables dérivées
df_finale = fe.creer_variables_derivees(df_finale)

print("\nDimensions de la base finale:", df_finale.shape)
print("\nAperçu de la base finale:")
print(df_finale.head())

## 2. Analyse Descriptive Complète

### 2.1 Distribution de la fréquence des sinistres

In [ ]:
# Distribution de la variable cible (fréquence)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Indicateur binaire
df_finale['a_sinistre'].value_counts().plot(kind='bar', ax=axes[0])
axes[0].set_title('Distribution de la Sinistralité (0/1)')
axes[0].set_xlabel('A eu un sinistre')
axes[0].set_ylabel('Nombre de polices')

# Nombre de sinistres
df_finale[df_finale['nb_sinistres_total'] > 0]['nb_sinistres_total'].hist(bins=20, ax=axes[1])
axes[1].set_title('Distribution du Nombre de Sinistres (si > 0)')
axes[1].set_xlabel('Nombre de sinistres')
axes[1].set_ylabel('Fréquence')

plt.tight_layout()
plt.show()

print(f"Taux de sinistralité: {df_finale['a_sinistre'].mean()*100:.2f}%")
print(f"Nombre moyen de sinistres (si sinistre): {df_finale[df_finale['nb_sinistres_total'] > 0]['nb_sinistres_total'].mean():.2f}")

### 2.2 Distribution de la gravité des sinistres

In [ ]:
# Analyser uniquement les polices avec sinistre
df_avec_sinistre = df_finale[df_finale['a_sinistre'] == 1]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution du montant total
df_avec_sinistre['montant_total'].hist(bins=50, ax=axes[0])
axes[0].set_title('Distribution du Montant Total des Sinistres')
axes[0].set_xlabel('Montant (€)')
axes[0].set_ylabel('Fréquence')

# Distribution log du montant
np.log1p(df_avec_sinistre['montant_total']).hist(bins=50, ax=axes[1])
axes[1].set_title('Distribution Log du Montant Total')
axes[1].set_xlabel('Log(Montant + 1)')
axes[1].set_ylabel('Fréquence')

plt.tight_layout()
plt.show()

print(f"Montant moyen des sinistres: {df_avec_sinistre['montant_total'].mean():.2f}€")
print(f"Montant médian: {df_avec_sinistre['montant_total'].median():.2f}€")
print(f"Montant max: {df_avec_sinistre['montant_total'].max():.2f}€")

### 2.3 Analyse par variables d'assurance

In [ ]:
# Sinistralité par couverture
sinistralite_par_couverture = df_finale.groupby('pol_coverage').agg({
    'a_sinistre': ['mean', 'count'],
    'montant_total': 'mean'
}).round(4)

print("Sinistralité par type de couverture:")
print(sinistralite_par_couverture)

# Visualisation
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Par usage
df_finale.groupby('pol_usage')['a_sinistre'].mean().plot(kind='bar', ax=axes[0])
axes[0].set_title('Taux de Sinistralité par Usage')
axes[0].set_ylabel('Taux')
axes[0].tick_params(axis='x', rotation=45)

# Par âge véhicule
if 'vh_age_cat' in df_finale.columns:
    df_finale.groupby('vh_age_cat')['a_sinistre'].mean().plot(kind='bar', ax=axes[1])
    axes[1].set_title('Taux de Sinistralité par Âge Véhicule')
    axes[1].set_ylabel('Taux')
    axes[1].tick_params(axis='x', rotation=45)

# Par âge conducteur
if 'drv_age1_cat' in df_finale.columns:
    df_finale.groupby('drv_age1_cat')['a_sinistre'].mean().plot(kind='bar', ax=axes[2])
    axes[2].set_title('Taux de Sinistralité par Âge Conducteur')
    axes[2].set_ylabel('Taux')
    axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 2.4 Corrélation entre variables

In [ ]:
# Sélectionner les variables numériques principales
vars_numeriques = ['pol_bonus', 'pol_duration', 'drv_age1', 'drv_age_lic1',
                   'vh_age', 'vh_din', 'vh_value', 'a_sinistre', 'nb_sinistres_total']
vars_disponibles = [v for v in vars_numeriques if v in df_finale.columns]

# Matrice de corrélation
corr_matrix = df_finale[vars_disponibles].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Matrice de Corrélation - Variables Principales')
plt.tight_layout()
plt.show()

### 2.5 Description des variables climatiques

In [ ]:
# Identifier les variables climatiques dans la base finale
cols_climat = [col for col in df_finale.columns 
               if any(prefix in col for prefix in ['RR', 'TX', 'TN', 'TM', 'UN', 'FF'])]

if len(cols_climat) > 0:
    print(f"Nombre de variables climatiques: {len(cols_climat)}")
    print("\nStatistiques descriptives des principales variables climatiques:")
    print(df_finale[cols_climat[:10]].describe())
    
    # Corrélation entre climat et sinistralité
    if len(cols_climat) >= 5:
        climat_vs_sinistre = df_finale[cols_climat[:20] + ['a_sinistre']].corr()['a_sinistre'].sort_values(ascending=False)
        print("\nTop 10 corrélations climat-sinistralité:")
        print(climat_vs_sinistre[1:11])  # Exclure la corrélation avec soi-même
else:
    print("Aucune variable climatique trouvée dans la base finale.")

## 3. Réduction de Dimension sur les Variables Climatiques

### 3.1 Préparation des données climatiques

In [ ]:
# Sélectionner uniquement les observations avec données climatiques
df_avec_climat = df_finale[cols_climat].dropna()

# Limiter aux variables climatiques les plus importantes (pour performance)
# Sélectionner max 50 variables
cols_climat_reduced = cols_climat[:min(50, len(cols_climat))]
X_climat = df_avec_climat[cols_climat_reduced]

print(f"Données climatiques pour ACP: {X_climat.shape}")
print(f"Variables sélectionnées: {len(cols_climat_reduced)}")

### 3.2 Standardisation

In [ ]:
# Standardiser les variables climatiques
X_climat_scaled, scaler_climat, feature_names_climat = dr.standardiser_variables(
    X_climat, 
    feature_names=cols_climat_reduced
)

print("Standardisation effectuée.")
print(f"Moyenne après standardisation: {X_climat_scaled.mean():.6f}")
print(f"Écart-type après standardisation: {X_climat_scaled.std():.2f}")

### 3.3 Analyse en Composantes Principales (ACP)

In [ ]:
# Effectuer l'ACP
resultats_acp = dr.analyse_acp(
    X_climat_scaled, 
    n_components=min(20, X_climat.shape[1]), 
    feature_names=feature_names_climat
)

# Visualiser la variance expliquée
fig = dr.visualiser_variance_expliquee(resultats_acp)
plt.show()

### 3.4 Interprétation des composantes

In [ ]:
# Visualiser les loadings
fig = dr.visualiser_loadings(resultats_acp['loadings'], n_components=5, n_vars=10)
plt.show()

# Interpréter les composantes
interpretations = dr.interpreter_composantes_climat(resultats_acp['loadings'], n_components=5)
for comp, interp in interpretations.items():
    print("\n" + "="*60)
    print(interp)

### 3.5 Partial Least Squares (PLS)

In [ ]:
# Préparer y pour PLS (sinistralité)
indices_climat = df_finale[cols_climat].dropna().index
y_climat = df_finale.loc[indices_climat, 'a_sinistre']

# Aligner X et y
X_pls = X_climat_scaled
y_pls = y_climat.values

# Effectuer la PLS
resultats_pls = dr.analyser_pls(
    X_pls, 
    y_pls, 
    n_components=10, 
    feature_names=feature_names_climat
)

# Visualiser les loadings PLS
fig = dr.visualiser_loadings(resultats_pls['loadings'], n_components=5, n_vars=10)
plt.show()

## 4. Modélisation - Fréquence des Sinistres

### 4.1 Préparation des données de modélisation

In [ ]:
# Sélectionner les features pour la modélisation
features = fe.selectionner_features_modelisation(df_finale, inclure_climat=True)

print(f"Nombre de features sélectionnées: {len(features)}")
print(f"Features: {features[:20]}...")  # Afficher les 20 premières

# Préparer X et y
X, y, indices = fe.preparer_donnees_modelisation(
    df_finale, 
    features, 
    target='a_sinistre'
)

### 4.2 Division train/test

In [ ]:
# Diviser les données
X_train, X_test, y_train, y_test = md.diviser_donnees(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

### 4.3 Modèles de classification

#### 4.3.1 Régression Logistique

In [ ]:
# Entraîner la régression logistique
model_logistic = md.entrainer_logistic_regression(X_train, y_train)

# Prédictions
y_pred_logistic = model_logistic.predict(X_test)
y_pred_proba_logistic = model_logistic.predict_proba(X_test)[:, 1]

# Évaluation
metriques_logistic = ev.evaluer_classification(y_test, y_pred_logistic, y_pred_proba_logistic)
ev.afficher_metriques_classification(metriques_logistic, "Régression Logistique")

#### 4.3.2 Régression Logistique Pénalisée (Lasso)

In [ ]:
# Entraîner Lasso
model_lasso = md.entrainer_logistic_penalisee(X_train, y_train, penalty='l1', C=0.1)

# Prédictions
y_pred_lasso = model_lasso.predict(X_test)
y_pred_proba_lasso = model_lasso.predict_proba(X_test)[:, 1]

# Évaluation
metriques_lasso = ev.evaluer_classification(y_test, y_pred_lasso, y_pred_proba_lasso)
ev.afficher_metriques_classification(metriques_lasso, "Lasso Logistique")

#### 4.3.3 Random Forest Classifier

In [ ]:
# Entraîner Random Forest
model_rf = md.entrainer_random_forest_classifier(X_train, y_train, n_estimators=100, max_depth=10)

# Prédictions
y_pred_rf = model_rf.predict(X_test)
y_pred_proba_rf = model_rf.predict_proba(X_test)[:, 1]

# Évaluation
metriques_rf = ev.evaluer_classification(y_test, y_pred_rf, y_pred_proba_rf)
ev.afficher_metriques_classification(metriques_rf, "Random Forest")

#### 4.3.4 XGBoost Classifier

In [ ]:
# Entraîner XGBoost
model_xgb = md.entrainer_xgboost_classifier(X_train, y_train, n_estimators=100, max_depth=5, learning_rate=0.1)

# Prédictions
y_pred_xgb = model_xgb.predict(X_test)
y_pred_proba_xgb = model_xgb.predict_proba(X_test)[:, 1]

# Évaluation
metriques_xgb = ev.evaluer_classification(y_test, y_pred_xgb, y_pred_proba_xgb)
ev.afficher_metriques_classification(metriques_xgb, "XGBoost")

### 4.4 Comparaison des modèles de fréquence

In [ ]:
# Compiler les résultats
resultats_frequence = {
    'Logistic Regression': metriques_logistic,
    'Lasso': metriques_lasso,
    'Random Forest': metriques_rf,
    'XGBoost': metriques_xgb
}

# Comparaison
df_comp_freq = ev.comparer_modeles(resultats_frequence, metrique='roc_auc')

# Visualisation
ev.visualiser_comparaison_modeles(resultats_frequence, 
                                   metriques_a_comparer=['accuracy', 'roc_auc', 'f1_score'])
plt.show()

### 4.5 Courbe ROC

In [ ]:
# Courbes ROC pour tous les modèles
plt.figure(figsize=(10, 8))

from sklearn.metrics import roc_curve, roc_auc_score

for nom, y_proba in [('Logistic', y_pred_proba_logistic), 
                      ('Lasso', y_pred_proba_lasso),
                      ('Random Forest', y_pred_proba_rf), 
                      ('XGBoost', y_pred_proba_xgb)]:
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)
    plt.plot(fpr, tpr, label=f'{nom} (AUC = {auc:.4f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('Taux de Faux Positifs')
plt.ylabel('Taux de Vrais Positifs')
plt.title('Courbes ROC - Comparaison des Modèles')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. Modélisation - Gravité des Sinistres

### 5.1 Préparation des données (seulement polices avec sinistre)

In [ ]:
# Filtrer uniquement les polices avec sinistre
df_gravite = df_finale[df_finale['a_sinistre'] == 1].copy()

# Préparer X et y pour la gravité
X_grav, y_grav, indices_grav = fe.preparer_donnees_modelisation(
    df_gravite,
    features,
    target='montant_total'
)

print(f"\nDonnées gravité: {X_grav.shape}")
print(f"Montant moyen: {y_grav.mean():.2f}€")

# Division train/test
X_train_g, X_test_g, y_train_g, y_test_g = md.diviser_donnees(
    X_grav, y_grav, test_size=0.2, random_state=42
)

### 5.2 Modèles de régression

#### 5.2.1 Régression Linéaire

In [ ]:
# Entraîner régression linéaire
model_linear = md.entrainer_regression_lineaire(X_train_g, y_train_g)

# Prédictions
y_pred_linear = model_linear.predict(X_test_g)

# Évaluation
metriques_linear = ev.evaluer_regression(y_test_g, y_pred_linear)
ev.afficher_metriques_regression(metriques_linear, "Régression Linéaire")

#### 5.2.2 Ridge Regression

In [ ]:
# Entraîner Ridge
model_ridge = md.entrainer_regression_penalisee(X_train_g, y_train_g, method='ridge', alpha=1.0)

# Prédictions
y_pred_ridge = model_ridge.predict(X_test_g)

# Évaluation
metriques_ridge = ev.evaluer_regression(y_test_g, y_pred_ridge)
ev.afficher_metriques_regression(metriques_ridge, "Ridge")

#### 5.2.3 Random Forest Regressor

In [ ]:
# Entraîner Random Forest
model_rf_reg = md.entrainer_random_forest_regressor(X_train_g, y_train_g, n_estimators=100, max_depth=10)

# Prédictions
y_pred_rf_reg = model_rf_reg.predict(X_test_g)

# Évaluation
metriques_rf_reg = ev.evaluer_regression(y_test_g, y_pred_rf_reg)
ev.afficher_metriques_regression(metriques_rf_reg, "Random Forest Regressor")

#### 5.2.4 XGBoost Regressor

In [ ]:
# Entraîner XGBoost
model_xgb_reg = md.entrainer_xgboost_regressor(X_train_g, y_train_g, n_estimators=100, max_depth=5, learning_rate=0.1)

# Prédictions
y_pred_xgb_reg = model_xgb_reg.predict(X_test_g)

# Évaluation
metriques_xgb_reg = ev.evaluer_regression(y_test_g, y_pred_xgb_reg)
ev.afficher_metriques_regression(metriques_xgb_reg, "XGBoost Regressor")

### 5.3 Comparaison des modèles de gravité

In [ ]:
# Compiler les résultats
resultats_gravite = {
    'Linear Regression': metriques_linear,
    'Ridge': metriques_ridge,
    'Random Forest': metriques_rf_reg,
    'XGBoost': metriques_xgb_reg
}

# Comparaison
df_comp_grav = ev.comparer_modeles(resultats_gravite, metrique='r2')

# Visualisation
ev.visualiser_comparaison_modeles(resultats_gravite, 
                                   metriques_a_comparer=['r2', 'mae', 'rmse'])
plt.show()

### 5.4 Visualisation des prédictions

In [ ]:
# Prédictions vs réelles pour le meilleur modèle
ev.graphique_predictions_vs_reelles(y_test_g, y_pred_xgb_reg)
plt.show()

# Résidus
ev.graphique_residus(y_test_g, y_pred_xgb_reg)
plt.show()

## 6. Interprétation des Résultats

### 6.1 Feature Importance - Fréquence

In [ ]:
# Feature importance pour Random Forest (fréquence)
importance_rf = md.extraire_feature_importance(model_rf, features)
print("\nTop 20 Features les plus importantes (Random Forest - Fréquence):")
print(importance_rf.head(20))

# Visualisation
ev.visualiser_feature_importance(importance_rf, top_n=20)
plt.show()

### 6.2 Feature Importance - Gravité

In [ ]:
# Feature importance pour Random Forest (gravité)
importance_rf_grav = md.extraire_feature_importance(model_rf_reg, features)
print("\nTop 20 Features les plus importantes (Random Forest - Gravité):")
print(importance_rf_grav.head(20))

# Visualisation
ev.visualiser_feature_importance(importance_rf_grav, top_n=20)
plt.show()

### 6.3 SHAP Values (optionnel)

In [ ]:
# Calculer les SHAP values pour le modèle Random Forest (fréquence)
try:
    shap_values = ev.calculer_shap_values(model_rf, X_test, feature_names=features, max_display=20)
    plt.show()
except Exception as e:
    print(f"SHAP values non calculées: {e}")

## 7. Conclusions et Recommandations pour un Assureur

### 7.1 Synthèse des performances

In [ ]:
print("="*80)
print("SYNTHÈSE DES RÉSULTATS")
print("="*80)

print("\n1. MODÉLISATION DE LA FRÉQUENCE DES SINISTRES")
print("-" * 80)
for nom, metriques in resultats_frequence.items():
    print(f"{nom:25s} - AUC: {metriques['roc_auc']:.4f}, Accuracy: {metriques['accuracy']:.4f}")

print("\n2. MODÉLISATION DE LA GRAVITÉ DES SINISTRES")
print("-" * 80)
for nom, metriques in resultats_gravite.items():
    print(f"{nom:25s} - R²: {metriques['r2']:.4f}, RMSE: {metriques['rmse']:.2f}€")

print("\n3. MEILLEUR MODÈLE PAR TÂCHE")
print("-" * 80)
best_freq = max(resultats_frequence.items(), key=lambda x: x[1]['roc_auc'])
best_grav = max(resultats_gravite.items(), key=lambda x: x[1]['r2'])
print(f"Fréquence : {best_freq[0]} (AUC = {best_freq[1]['roc_auc']:.4f})")
print(f"Gravité   : {best_grav[0]} (R² = {best_grav[1]['r2']:.4f})")

### 7.2 Recommandations opérationnelles

#### Facteurs de risque identifiés

Sur la base de l'analyse des features importance et des modèles, voici les principaux facteurs de risque :

**Facteurs assurantiels :**
- Bonus-malus du conducteur
- Âge du conducteur
- Expérience de conduite
- Âge et puissance du véhicule
- Usage du véhicule

**Facteurs climatiques :**
- Les variables climatiques ont été réduites en composantes principales
- Certaines composantes montrent une corrélation avec la sinistralité
- L'impact varie selon les régions et périodes

#### Applications pour l'assureur

1. **Tarification** : Ajuster les primes en fonction des facteurs identifiés
2. **Souscription** : Améliorer le processus de sélection des risques
3. **Prévention** : Cibler les actions de prévention sur les profils à risque
4. **Provisionnement** : Mieux estimer les réserves nécessaires

#### Limites et perspectives

- Les données climatiques pourraient être affinées (granularité temporelle)
- D'autres variables externes pourraient être intégrées (trafic, infrastructure)
- Les modèles pourraient être améliorés avec plus de données historiques

## 8. Sauvegarde des Résultats

In [ ]:
# Sauvegarder les principaux résultats
import pickle
import os

# Créer un dossier pour les résultats
os.makedirs('../results', exist_ok=True)

# Sauvegarder les modèles
with open('../results/best_model_frequence.pkl', 'wb') as f:
    pickle.dump(model_xgb, f)

with open('../results/best_model_gravite.pkl', 'wb') as f:
    pickle.dump(model_xgb_reg, f)

# Sauvegarder les résultats de comparaison
df_comp_freq.to_csv('../results/comparaison_modeles_frequence.csv', index=False)
df_comp_grav.to_csv('../results/comparaison_modeles_gravite.csv', index=False)

# Sauvegarder les feature importances
importance_rf.to_csv('../results/feature_importance_frequence.csv', index=False)
importance_rf_grav.to_csv('../results/feature_importance_gravite.csv', index=False)

print("Résultats sauvegardés dans le dossier 'results/'")

---

## Fin du Notebook

Ce notebook a présenté une analyse complète de l'impact des conditions climatiques sur la sinistralité automobile, incluant :

1. ✅ Construction de la base jointe
2. ✅ Analyse descriptive
3. ✅ Réduction de dimension (ACP/PLS)
4. ✅ Modélisation de la fréquence
5. ✅ Modélisation de la gravité
6. ✅ Comparaison et sélection des modèles
7. ✅ Interprétation des résultats